### Knock51: Data 加工の下準備をしよう
- Data 加工の下準備に入る前に、どのような機械学習 Model を構築するかを考える。
- 今回は、教師あり学習（二値分類）Model
    - 前月までの Data から、来月の Order 数が増加するのか減少するのか予測する Case は、Order 数自体を予測するのではなく、増加するのか減少するのか減少かを予測する。

In [1]:
# Directory 作成
import os

data_dir = 'data'
input_dir = os.path.join(data_dir, '0_input')
output_dir = os.path.join(data_dir, '1_output')
master_dir = os.path.join(data_dir, '99_master')
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)

Data を読み込む際に File 複数が存在する場合、一つ一つ指定していたら大変なので、自動的に Directory の中身を取りに行き、配列として Path を定義する

In [2]:
# 注文 Data path の取得
import glob
tbl_order_file = os.path.join(input_dir, 'tbl_order_*.csv')
tbl_order_path = glob.glob(tbl_order_file)
tbl_order_path

['data\\0_input\\tbl_order_201904.csv',
 'data\\0_input\\tbl_order_201905.csv',
 'data\\0_input\\tbl_order_201906.csv',
 'data\\0_input\\tbl_order_201907.csv',
 'data\\0_input\\tbl_order_201908.csv',
 'data\\0_input\\tbl_order_201909.csv',
 'data\\0_input\\tbl_order_201910.csv',
 'data\\0_input\\tbl_order_201911.csv',
 'data\\0_input\\tbl_order_201912.csv',
 'data\\0_input\\tbl_order_202001.csv',
 'data\\0_input\\tbl_order_202002.csv',
 'data\\0_input\\tbl_order_202003.csv']

### Knock52: Data の読み込みを行ない加工の方向性を検討しよう

In [3]:
# Master data の読み込み
import pandas as pd

m_area_file = 'm_area.csv'
m_store_file = 'm_store.csv'
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
m_area.head(3)

,area_cd,wide_area,narrow_area
0,TK,東京,東京
1,KN,神奈川,神奈川
2,CH,千葉,千葉


Directory 構造をしっかり考え、変数名を統一することで、簡単に使い回すことができる。

In [4]:
# 注文 Data の読み込み
tbl_order_path = tbl_order_path[0]
print(f"読み込み Data: {tbl_order_path}")
order_data = pd.read_csv(tbl_order_path)
print(f"Data 件数: {len(order_data)}")
order_data.head(3)

読み込み Data: data\0_input\tbl_order_201904.csv
Data 件数: 233393


,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status
0,22808272,39,C80973292,40,22222408,2019-04-01 11:00:00,2019-04-01 11:26:00,1,2112,1
1,10902625,63,C94948343,20,79467084,2019-04-01 11:00:00,2019-04-01 11:47:00,0,2154,2
2,5990375,63,C91814442,1,61749935,2019-04-01 11:00:00,2019-04-01 11:10:00,0,3050,2


Data 件数が非常に多い（7桁くらい）場合、Data をやり取りするには、処理に時間がかかってしまう。
よって、まずは少量の Data 量で加工の方向を考え、全期間の Data に適用する。

### Knock53: 1ヶ月分の Data の基本的な Data 加工を実施しよう

In [5]:
# 基本的な Data 加工
order_data = order_data.loc[order_data['store_id'] != 999]

order_data = pd.merge(order_data, m_store, on='store_id', how='left')
order_data = pd.merge(order_data, m_area, on='area_cd', how='left')

order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = "デリバリー"
order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = "お持ち帰り"

order_data.loc[order_data['status'] == 0, 'status_name'] = "受付"
order_data.loc[order_data['status'] == 1, 'status_name'] = "お支払済"
order_data.loc[order_data['status'] == 2, 'status_name'] = "お渡し済"
order_data.loc[order_data['status'] == 9, 'status_name'] = "キャンセル"
order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name
0,22808272,39,C80973292,40,22222408,2019-04-01 11:00:00,2019-04-01 11:26:00,1,2112,1,杉並店,TK,東京,東京,お持ち帰り,お支払済
1,10902625,63,C94948343,20,79467084,2019-04-01 11:00:00,2019-04-01 11:47:00,0,2154,2,西多摩店,TK,東京,東京,デリバリー,お渡し済
2,5990375,63,C91814442,1,61749935,2019-04-01 11:00:00,2019-04-01 11:10:00,0,3050,2,西多摩店,TK,東京,東京,デリバリー,お渡し済


In [6]:
# 欠損値の確認
order_data.isna().sum()

order_id             0
store_id             0
customer_id          0
coupon_cd            0
sales_detail_id      0
order_accept_date    0
delivered_date       0
takeout_flag         0
total_amount         0
status               0
store_name           0
area_cd              0
wide_area            0
narrow_area          0
takeout_name         0
status_name          0
dtype: int64

### Knock54: 機械学習に使用する変数を作成しよう

In [7]:
# Pizza 提供までの時間の作成
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds() / 60


order_data.loc[:, 'order_accept_datetime'] = pd.to_datetime(order_data['order_accept_date'])
order_data.loc[:, 'delivered_datetime'] = pd.to_datetime(order_data['delivered_date'])
order_data.loc[:, 'delta'] = order_data[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis=1)
order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name,order_accept_datetime,delivered_datetime,delta
0,22808272,39,C80973292,40,22222408,2019-04-01 11:00:00,2019-04-01 11:26:00,1,2112,1,杉並店,TK,東京,東京,お持ち帰り,お支払済,2019-04-01 11:00:00,2019-04-01 11:26:00,26.0
1,10902625,63,C94948343,20,79467084,2019-04-01 11:00:00,2019-04-01 11:47:00,0,2154,2,西多摩店,TK,東京,東京,デリバリー,お渡し済,2019-04-01 11:00:00,2019-04-01 11:47:00,47.0
2,5990375,63,C91814442,1,61749935,2019-04-01 11:00:00,2019-04-01 11:10:00,0,3050,2,西多摩店,TK,東京,東京,デリバリー,お渡し済,2019-04-01 11:00:00,2019-04-01 11:10:00,10.0


In [8]:
# 日付関連変数の作成
order_data.loc[:, 'order_accept_hour'] = order_data['order_accept_datetime'].dt.hour
order_data.loc[:, 'order_accept_weekday'] = order_data['order_accept_datetime'].dt.weekday
order_data.loc[order_data['order_accept_weekday'] >= 5, 'weekday_info'] = "休日"
order_data.loc[order_data['order_accept_weekday'] < 5, 'weekday_info'] = "平日"
order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,...,wide_area,narrow_area,takeout_name,status_name,order_accept_datetime,delivered_datetime,delta,order_accept_hour,order_accept_weekday,weekday_info
0,22808272,39,C80973292,40,22222408,2019-04-01 11:00:00,2019-04-01 11:26:00,1,2112,1,...,東京,東京,お持ち帰り,お支払済,2019-04-01 11:00:00,2019-04-01 11:26:00,26.0,11,0,平日
1,10902625,63,C94948343,20,79467084,2019-04-01 11:00:00,2019-04-01 11:47:00,0,2154,2,...,東京,東京,デリバリー,お渡し済,2019-04-01 11:00:00,2019-04-01 11:47:00,47.0,11,0,平日
2,5990375,63,C91814442,1,61749935,2019-04-01 11:00:00,2019-04-01 11:10:00,0,3050,2,...,東京,東京,デリバリー,お渡し済,2019-04-01 11:00:00,2019-04-01 11:10:00,10.0,11,0,平日


- weekday は、0 の月曜日に始まり、6 が日曜日となる。
- 上記より、平日: 5 未満、休日: 5 以上で分岐できる。